<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Connect-to-workspace" data-toc-modified-id="Connect-to-workspace-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Connect to workspace</a></span></li><li><span><a href="#Define-Datastore" data-toc-modified-id="Define-Datastore-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Define Datastore</a></span><ul class="toc-item"><li><span><a href="#Up-load-file-(folder)-to-Datastore" data-toc-modified-id="Up-load-file-(folder)-to-Datastore-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Up load file (folder) to Datastore</a></span></li></ul></li></ul></div>

In [1]:
import azureml.core
from azureml.core import Workspace, Run, Dataset

In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment, ScriptRunConfig, Experiment
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

In [3]:
import os

## Connect to workspace

In [5]:
#import workspace from config.json
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.35.0 to work with projet_7


## Define Datastore

In [7]:
default_ds = ws.get_default_datastore()
print(default_ds)

{
  "name": "dataset_projet7",
  "container_name": "dataset-projet7",
  "account_name": "stockageprojet7opcr",
  "protocol": "https",
  "endpoint": "core.windows.net"
}


### Up load file (folder) to Datastore

In [11]:
filepath = os.path.abspath("gtfine")
filepath

'C:\\Users\\favre\\projet_9\\gtfine'

In azureCLI:

az storage blob upload-batch --destination dataset-projet7 --source C:\\Users\\favre\\projet_9\\photo --account-key ujARUiYsDABZ9UTCzWwzdwaNDscT3dIUsZ/EMvCtx38XhSvZ5Q5Ks5hKn8YnGc5Q+1P9jrMmLR6vl70nQdzrBA==  --account-name stockageprojet7opcr  

## up date env from yml

conda env update --name myenv --file local.yml --prune  
conda env update --name proj9 --file env-p9.yml --prune 
conda env update -p d:\\anaconda\\envs -f env-p9 --prune
  
 --prune uninstalls dependencies which were removed from local.yml

## Compute cluster

In [6]:
cluster_name = 'cluster-projet7'

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


## Create environment

In [23]:
#create environment from yml file
env_p9 = Environment.from_conda_specification("env_p9", 'training_azure/env-p9.yml')

No Python version provided, defaulting to "3.6.2"


## Script

In [13]:
%%writefile training_azure/data_augmentation.py
print('print importing lib...')

Writing training_azure/data_augmentation.py


In [19]:
%%writefile training_azure/training.py
print('print importing lib...')

import argparse
from azureml.core import Run
from azureml.core import Dataset
import joblib
import os
import numpy as np
import pandas as pd
import random
import joblib
import cv2

from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from tensorflow.python.keras.optimizers import Adadelta, Nadam
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.utils import multi_gpu_model, plot_model
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.losses import binary_crossentropy
from tensorflow.python.keras.utils import Sequence
from tensorflow.python.keras.callbacks import Callback

from dilatednet import DilatedNet
from multiclassunet import Unet

print('lib imported...')

#get scripts arguments
parser = argparse.ArgumentParser()
parser.add_argument('--image-folder', type=str, dest='image_folder')
parser.add_argument('--mask-folder', type=, dest='mask_folder')

#set parameters
batch_size = 2
samples = 50000
steps = samples//batch_size
img_height, img_width = 256, 256
classes = 8
filters_n = 64

#load data
print('data...')
mask_path = run.input_datasets['mask']
image_path = run.input_datasets['image']

#divison of labelling
cats = {'void': [0, 1, 2, 3, 4, 5, 6],
 'flat': [7, 8, 9, 10],
 'construction': [11, 12, 13, 14, 15, 16],
 'object': [17, 18, 19, 20],
 'nature': [21, 22],
 'sky': [23],
 'human': [24, 25],
 'vehicle': [26, 27, 28, 29, 30, 31, 32, 33, -1]}

#mounted point to directory
image_dir = mask_path
mask_dir = image_path
mask = 'labelIds'

#extract list of pictures 
print('making picture list...')
image_list = []
for root, dirs, files in os.walk(image_dir):
    for name in files:
        image_list.append(os.path.join(root, name))

#extract mask with label
mask_list_l = []
for root, dirs, files in os.walk(mask_dir):
    for name in files:
        mask_list_l.append(os.path.join(root, name))

mask_list = [m for m in mask_list_l if mask in m]

print("class gen...")

class seg_gen(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        #make a array of length batch size containing random position of photo
        idx = np.random.randint(0, 10, batch_size)
        batch_x, batch_y = [], []
        drawn = 0
        
        for i in idx:
            #import original image /255 to normalize and avoid big number for compute power
            _image = image.img_to_array(image.load_img(f'{image_list[i]}', target_size=(img_height, img_width)))/255.
            #import mask
            img = image.img_to_array(image.load_img(f'{mask_list[i]}', grayscale=True, target_size=(img_height, img_width)))
            #assess number of different label
            labels = np.unique(img)
            #ignore picture withtout features
            if len(labels) < 3:
                idx = np.random.randint(0, 50000, batch_size-drawn)
                continue
            img = np.squeeze(img)
            #create a tensor of dimension image with one plan for each category
            mask = np.zeros((img.shape[0], img.shape[1], 8))
            for i in range(-1, 34):
                if i in cats['void']:
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i in cats['flat']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i in cats['construction']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i in cats['object']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i in cats['nature']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i in cats['sky']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                elif i in cats['human']:
                    mask[:,:,6] = np.logical_or(mask[:,:,6],(img==i))
                elif i in cats['vehicle']:
                    mask[:,:,7] = np.logical_or(mask[:,:,7],(img==i))
            mask = np.resize(mask,(img_height*img_width, 8))
            batch_y.append(mask)
            batch_x.append(_image)
            drawn += 1
        return np.array(batch_x), np.array(batch_y)
    
print('coef and loss...')

def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def total_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + (3*dice_loss(y_true, y_pred))
    return loss\

print('model...')

unet = DilatedNet(256, 256, 8,use_ctx_module=True, bn=True)
unet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[dice_coeff, 'accuracy'])
tb = TensorBoard(log_dir='logs', write_graph=True)
mc = ModelCheckpoint(mode='max', filepath='models/pdilated.h5', monitor='acc', save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='acc', patience=6, verbose=1)
callbacks = [tb, mc, es]

print('making input...')
train_gen = seg_gen(image_list[:10], mask_list[:10], batch_size)

print('fit...')
unet.fit_generator(train_gen, epochs=5, callbacks=callbacks)

print('Saving final weights')
os.makedirs('outputs', exist_ok=True)
model_file = os.path.join('outputs', 'dilated.h5')
unet.save_weights(model_file)

run.complete()

Overwriting training_azure/training.py


# Pipeline

## Configuration

In [12]:
from azureml.core import Environment
from azureml.core.runconfig import RunConfiguration

#experiment folder
experiment_folder = 'training_azure'

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

#assign compute
pipeline_run_config.target = training_cluster

# Assign the environment to the run configuration
pipeline_run_config.environment = env_p9

## Step

In [15]:
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.steps import PythonScriptStep

# Get the training dataset
image_ds = ws.datasets.get("image")
mask_ds = ws.datasets.get('cityscape_train')

# Create an OutputFileDatasetConfig (temporary Data Reference) for data passed from step 1 to step 2
#with no specification in OutputFileDatasetConfig for destination it will be in workspaceblobstore datastore
augmented_data = OutputFileDatasetConfig("augmented_data")

#step1, run the data prep script
prep_step = PythonScriptStep(name = "Data augmentation",
                                source_directory = experiment_folder,
                                script_name = "data_augmentation.py",
                                arguments = ['--augmented_data', augmented_data,
                                             '--input-data', image_ds.as_named_input('image').as_mount(),
                                            '--input-data2', mask_ds.as_named_input('mask').as_mount()],
                                compute_target = training_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

# Step 2, run the training script
train_step = PythonScriptStep(name = "Train and Register Model",
                                source_directory = experiment_folder,
                                script_name = "training.py",
                                arguments = ['--training-data', augmented_data.as_input()],
                                compute_target = training_cluster,
                                runconfig = pipeline_run_config,
                                allow_reuse = True)

## Build pipeline

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

# Construct the pipeline
pipeline_steps = [prep_step, train_step]
pipeline = Pipeline(workspace=ws, steps=pipeline_steps)


# Create an experiment and run the pipeline
experiment = Experiment(workspace=ws, name = 'recommender-pipeline')
pipeline_run = experiment.submit(pipeline, regenerate_outputs=True)


pipeline_run.wait_for_completion(show_output=True)

## Examine 

see projet 8

## Script testing

In [20]:
experiment_name = 'image-segmentation-unet'

from azureml.core import Experiment
exp = Experiment(workspace=ws, name=experiment_name)

In [25]:
# Get the training dataset
image_ds = ws.datasets.get("image")
mask_ds = ws.datasets.get('mask')

In [26]:
from azureml.core import ScriptRunConfig

args = ['--image-folder', image_ds.as_named_input('image').as_mount(),
        '--mask-folder', mask_ds.as_named_input('mask').as_mount()]

src = ScriptRunConfig(source_directory="training_azure",
                      script='training.py', 
                      arguments=args,
                      compute_target=training_cluster,
                      environment=env_p9)

In [27]:
run = exp.submit(config=src)
run.wait_for_completion(show_output=True)

RunId: image-segmentation-unet_1635443357_62f93dbd
Web View: https://ml.azure.com/runs/image-segmentation-unet_1635443357_62f93dbd?wsid=/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourcegroups/Projet_7/workspaces/projet_7&tid=33e47288-d1e1-43e8-b65b-4ba7bfd37a9f

Streaming azureml-logs/20_image_build_log.txt

2021/10/28 17:49:26 Downloading source code...
2021/10/28 17:49:26 Finished downloading source code
2021/10/28 17:49:27 Creating Docker network: acb_default_network, driver: 'bridge'
2021/10/28 17:49:27 Successfully set up Docker network: acb_default_network
2021/10/28 17:49:27 Setting up Docker configuration...
2021/10/28 17:49:28 Successfully set up Docker configuration
2021/10/28 17:49:28 Logging in to registry: 38d02f2005824c85859d3a5c11c9a143.azurecr.io
2021/10/28 17:49:28 Successfully logged into 38d02f2005824c85859d3a5c11c9a143.azurecr.io
2021/10/28 17:49:28 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
20


Execution Summary
RunId: image-segmentation-unet_1635443357_62f93dbd
Web View: https://ml.azure.com/runs/image-segmentation-unet_1635443357_62f93dbd?wsid=/subscriptions/b9053cbf-be55-4e83-8c03-d6b0eb90cb5a/resourcegroups/Projet_7/workspaces/projet_7&tid=33e47288-d1e1-43e8-b65b-4ba7bfd37a9f


ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt.",
        "messageFormat": "Image build failed. For more details, check log file {ArtifactPath}.",
        "messageParameters": {
            "ArtifactPath": "azureml-logs/20_image_build_log.txt"
        },
        "details": [],
        "innerError": {
            "code": "BadArgument",
            "innerError": {
                "code": "ImageBuildFailure"
            }
        }
    },
    "correlation": {
        "operation": "891cbde40faf7644bd6284670b91e5e4",
        "request": "efc80990c64b9352"
    },
    "environment": "westeurope",
    "location": "westeurope",
    "time": "2021-10-28T19:27:53.046434Z",
    "componentName": "execution-worker"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"err